<a href="https://colab.research.google.com/github/AdnaneMOUZAOUI/Analyse-de-sentiments-machine-learning/blob/Reviews-Analysis/Analyse_showroom_Bag_of_words_multi_ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importer le fichier depuis le drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import re
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, f_regression, mutual_info_regression, RFE, RFECV
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score, classification_report

import time
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

from sklearn.svm import SVC

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
start_programme = time.time()

In [ ]:
# Pour associer l'environnement avec Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Drive non connecté")

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab_supply_chain/Data/data_preparee.csv', index_col=0)
df.head()

,client,star,date_commande,date,nbr_avis,Commentaire,pays,Titre,longueur,majuscule,...,"ngrams_3_titre('tres', 'bon', 'site').1","ngrams_3_titre('bon', 'traitement', 'commande').1","ngrams_3_titre('tout', 'très', 'bien').1","ngrams_3_titre('peut', 'mieux', 'faire').1","ngrams_3_titre('très', 'bon', 'service').1","ngrams_3_titre('livraison', 'peu', 'long').1","ngrams_3_titre('livraison', 'non', 'respecté').1","ngrams_3_titre('délai', 'trop', 'long').1","ngrams_3_titre('très', 'bonne', 'qualité').1","ngrams_3_titre('produit', 'non', 'conforme').1"
0,Agani,1.0,: 26 août 2022,2023-02-27,2.0,"['jamais', 'reçu', 'commande', 'écris', 'showr...",FR,"['jamais', 'reçu', 'commande']",427,7,...,0,0,0,0,0,0,0,0,0,0
1,CHANTAL SLATKINE,1.0,: 26 février 2023,2023-02-27,2.0,"['commandé', 'collier', 'lun', 'ok', 'lautre',...",FR,"['commandé', 'collier']",537,13,...,0,0,0,0,0,0,0,0,0,0
2,LDC,1.0,: 26 février 2023,2023-02-27,2.0,"['commandé', 'airpods', 'reconditionnés', 'dit...",FR,"['très', 'déçue', 'dernière', 'commande']",563,8,...,0,0,0,0,0,0,0,0,0,0
3,anass jeffal,1.0,: 27 octobre 2022,2023-02-27,1.0,"['produit', 'acheté', 'septembre', 'retourné',...",FR,"['produit', 'acheté', 'septembre']",612,6,...,0,0,0,0,0,0,0,0,0,0
4,aurélien,1.0,: 27 février 2023,2023-02-27,1.0,"['encore', 'commande', 'partiellement', 'annul...",FR,"['encore', 'commande']",405,2,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['star'].unique()

array([ 1.,  5.,  3.,  4.,  2., nan])

In [ ]:
df.dropna(inplace=True)
print(df.isna().sum())


client                                              0
star                                                0
date_commande                                       0
date                                                0
nbr_avis                                            0
                                                   ..
ngrams_3_titre('livraison', 'peu', 'long').1        0
ngrams_3_titre('livraison', 'non', 'respecté').1    0
ngrams_3_titre('délai', 'trop', 'long').1           0
ngrams_3_titre('très', 'bonne', 'qualité').1        0
ngrams_3_titre('produit', 'non', 'conforme').1      0
Length: 221, dtype: int64


## Application d'un modèle Randomforest de classification sur les notes ( 1,2,3,4,5)


In [ ]:
## Application d'un modèle de classification randomforest sur les notes ( 1,2,3,4,5)

X = df.Commentaire

y = df.star

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french') )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()
t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()

print('le temps de calcul du randomforest classification est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score en randomforest pour les notes (1,2,3,4,5) est :', round(rf.score(X_test, y_test),2))
print()

# Affichage d'un rapport de classification
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print()
confusion_matrix


Vocabulary size:  4000
{'chaussures': 503, 'très': 3744, 'mauvaises': 2211, 'qualité': 2963, 'elles': 1264, 'trop': 3730, 'rapidement': 3021, 'livraison': 2049, 'temp': 3623, 'emballage': 1266, 'nickel': 2394, 'marchandises': 2178, 'conformes': 686, 'parfait': 2543, 'recommande': 3060, 'achats': 37, 'retours': 3213, 'faciles': 1425, 'produits': 2840, 'parfois': 2547}

le temps de calcul du randomforest classification est :  10.5 minutes

le score en randomforest pour les notes (1,2,3,4,5) est : 0.55

              precision    recall  f1-score   support

         1.0       0.53      0.62      0.57      4363
         2.0       0.31      0.06      0.10      3114
         3.0       0.44      0.47      0.46      6039
         4.0       0.49      0.44      0.46      8697
         5.0       0.65      0.77      0.70     11566

    accuracy                           0.55     33779
   macro avg       0.48      0.47      0.46     33779
weighted avg       0.52      0.55      0.53     33779





Classe prédite,1.0,2.0,3.0,4.0,5.0
Classe réelle,,,,,
1.0,2726,198,924,219,296
2.0,1126,197,1247,282,262
3.0,956,184,2855,1337,707
4.0,209,44,1111,3820,3513
5.0,125,18,341,2203,8879


In [ ]:
X_train.shape

(135113, 4000)

## Application du modèle RandomForest sur les notes (-1, 0, +1)
## pour signifier respectivement, négatif(1* ,2* ), neutre(3* ) et positif(4* ,5* )

In [ ]:
## Application du modèle RandomForest sur les notes (-1, 0, +1)
## pour signifier respectivement, négatif(1*,2*), neutre(3*) et positif(4*,5*)

X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [-1,-1,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french') )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))
print()

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})

t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()

print('le temps de calcul du randomforest classification, notes(-1, 0, +1) est : ', round((t1-t0)/60, 2), 'minutes')
print()
print('le score en randomforest est :', round(rf.score(X_test, y_test),2))
print()
# Affichage d'un rapport de classification
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print()
confusion_matrix


Vocabulary size:  4000

{'chaussures': 503, 'très': 3744, 'mauvaises': 2211, 'qualité': 2963, 'elles': 1264, 'trop': 3730, 'rapidement': 3021, 'livraison': 2049, 'temp': 3623, 'emballage': 1266, 'nickel': 2394, 'marchandises': 2178, 'conformes': 686, 'parfait': 2543, 'recommande': 3060, 'achats': 37, 'retours': 3213, 'faciles': 1425, 'produits': 2840, 'parfois': 2547}
le temps de calcul du randomforest classification, notes(-1, 0, +1) est :  6.75 minutes

le score en randomforest est : 0.77

              precision    recall  f1-score   support

        -1.0       0.67      0.74      0.70      7477
         0.0       0.52      0.24      0.33      6039
         1.0       0.84      0.93      0.88     20263

    accuracy                           0.77     33779
   macro avg       0.67      0.64      0.64     33779
weighted avg       0.74      0.77      0.74     33779





Classe prédite,-1.0,0.0,1.0
Classe réelle,,,
-1.0,5556,686,1235
0.0,2095,1444,2500
1.0,695,664,18904


## Application du modèle RandomForest sur les notes (0, 1)
## pour signifier respectivement, négatif(1* ,2* ,3* ,4* ) et positif(5* )

In [ ]:
# Application du modèle RandomForest sur les notes (0, 1)
# pour signifier respectivement, négatif(1*,2*,3*,4*) et positif(5*)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,0,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french') )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()


t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()


print('le temps de calcul du randomforest classification est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score en randomforest est :', round(rf.score(X_test, y_test),2))
print()

# Affichage d'un rapport de classification
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print()
confusion_matrix


Vocabulary size:  4000
{'chaussures': 503, 'très': 3744, 'mauvaises': 2211, 'qualité': 2963, 'elles': 1264, 'trop': 3730, 'rapidement': 3021, 'livraison': 2049, 'temp': 3623, 'emballage': 1266, 'nickel': 2394, 'marchandises': 2178, 'conformes': 686, 'parfait': 2543, 'recommande': 3060, 'achats': 37, 'retours': 3213, 'faciles': 1425, 'produits': 2840, 'parfois': 2547}

le temps de calcul du randomforest classification est :  7.64 minutes

le score en randomforest est : 0.79

              precision    recall  f1-score   support

         0.0       0.83      0.86      0.85     22213
         1.0       0.71      0.67      0.69     11566

    accuracy                           0.79     33779
   macro avg       0.77      0.76      0.77     33779
weighted avg       0.79      0.79      0.79     33779





Classe prédite,0.0,1.0
Classe réelle,,
0.0,19118,3095
1.0,3843,7723


## Application du modèle RandomForest sur les notes (0, 1)
## pour signifier respectivement, négatif (1* ,2* ,3* ) et positif(4* ,5* )

In [ ]:
# Application du modèle RandomForest sur les notes (0, 1)
# pour signifier respectivement, négatif (1*,2*,3*) et positif(4*,5*)

X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french') )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()

print('le temps de calcul du randomforest classification est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score en randomforest est :', round(rf.score(X_test, y_test),2))
print()

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print()
confusion_matrix


Vocabulary size:  4000
{'chaussures': 503, 'très': 3744, 'mauvaises': 2211, 'qualité': 2963, 'elles': 1264, 'trop': 3730, 'rapidement': 3021, 'livraison': 2049, 'temp': 3623, 'emballage': 1266, 'nickel': 2394, 'marchandises': 2178, 'conformes': 686, 'parfait': 2543, 'recommande': 3060, 'achats': 37, 'retours': 3213, 'faciles': 1425, 'produits': 2840, 'parfois': 2547}

le temps de calcul du randomforest classification est :  5.69 minutes

le score en randomforest est : 0.87

              precision    recall  f1-score   support

         0.0       0.84      0.83      0.83     13516
         1.0       0.89      0.89      0.89     20263

    accuracy                           0.87     33779
   macro avg       0.86      0.86      0.86     33779
weighted avg       0.87      0.87      0.87     33779





Classe prédite,0.0,1.0
Classe réelle,,
0.0,11173,2343
1.0,2197,18066


## Application d'un Gradient boost classifier sur les notes (0,1)
## pour signifier respectivement, négatif (1* ,2* ,3* ) et positif(4* ,5* )

In [ ]:
## Application d'un Gradient boost classifier sur les notes (0,1)
## pour signifier respectivement, négatif (1* ,2* ,3* ) et positif(4* ,5* )

X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french') )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

GB_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
GB_clf.fit(X_train, y_train)
y_pred = GB_clf.predict(X_test)

t1 = time.time()

print('le temps de calcul du GradientBoost classification est : ', round((t1-t0)/60,2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score du gradient boosting est :', round(GB_clf.score(X_test, y_test),2))
print()

# Affichage d'un rapport de classification

print(classification_report(y_test, y_pred))
print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print()
confusion_matrix


Vocabulary size:  4000
{'chaussures': 503, 'très': 3744, 'mauvaises': 2211, 'qualité': 2963, 'elles': 1264, 'trop': 3730, 'rapidement': 3021, 'livraison': 2049, 'temp': 3623, 'emballage': 1266, 'nickel': 2394, 'marchandises': 2178, 'conformes': 686, 'parfait': 2543, 'recommande': 3060, 'achats': 37, 'retours': 3213, 'faciles': 1425, 'produits': 2840, 'parfois': 2547}

le temps de calcul du GradientBoost classification est :  0.28 minutes

le score du gradient boosting est : 0.8

              precision    recall  f1-score   support

         0.0       0.84      0.62      0.71     13516
         1.0       0.78      0.92      0.85     20263

    accuracy                           0.80     33779
   macro avg       0.81      0.77      0.78     33779
weighted avg       0.81      0.80      0.79     33779





Classe prédite,0.0,1.0
Classe réelle,,
0.0,8325,5191
1.0,1545,18718


## Application de XGBOOST sur le ngram=1 sur les notes (0,1)

In [ ]:
X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french') )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=1000,learning_rate=0.01,max_depth=6,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en unigram est est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score de xgbooost en unigram est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )
print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print()
confusion_matrix




Vocabulary size:  4000
{'chaussures': 503, 'très': 3744, 'mauvaises': 2211, 'qualité': 2963, 'elles': 1264, 'trop': 3730, 'rapidement': 3021, 'livraison': 2049, 'temp': 3623, 'emballage': 1266, 'nickel': 2394, 'marchandises': 2178, 'conformes': 686, 'parfait': 2543, 'recommande': 3060, 'achats': 37, 'retours': 3213, 'faciles': 1425, 'produits': 2840, 'parfois': 2547}

le temps de calcul du Xgboost en unigram est est :  2.91 minutes

le score de xgbooost en unigram est : 0.83

              precision    recall  f1-score   support

         0.0       0.85      0.70      0.77     13516
         1.0       0.82      0.92      0.87     20263

    accuracy                           0.83     33779
   macro avg       0.84      0.81      0.82     33779
weighted avg       0.83      0.83      0.83     33779





Classe prédite,0,1
Classe réelle,,
0.0,9474,4042
1.0,1658,18605


  # n-gram = 2

## Application du bag of words en n-gram = 2

## Application du modèle RandomForest sur les notes (0, 1)
## pour signifier respectivement, négatif (1* ,2* ,3* ) et positif(4* ,5* )


In [ ]:
# Application du modèle RandomForest sur les notes (0, 1)
# pour signifier respectivement, négatif (1*,2*,3*) et positif(4*,5*)

X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(2,2) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})


t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()

print('le temps de calcul du randomforest classification en Bi-gram est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score en randomforest est :', round(rf.score(X_test, y_test),2))
print()

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

Vocabulary size:  4000
{'chaussures très': 583, 'très mauvaises': 3863, 'mauvaises qualité': 2101, 'livraison temp': 1919, 'temp emballage': 3581, 'emballage nickel': 1422, 'qualité produits': 2842, 'jaime bien': 1655, 'bien site': 385, 'rapide simple': 2933, 'tout bien': 3668, 'bien passé': 362, 'satisfaite produit': 3265, 'point relais': 2560, 'relais non': 3006, 'non disponible': 2232, 'cela fait': 523, 'deux fois': 1205, 'faire livrer': 1504, 'livrer colis': 1934}
le temps de calcul du randomforest classification en Bi-gram est :  5.08 minutes

le score en randomforest est : 0.82

              precision    recall  f1-score   support

         0.0       0.81      0.70      0.75     13516
         1.0       0.82      0.89      0.85     20263

    accuracy                           0.82     33779
   macro avg       0.82      0.80      0.80     33779
weighted avg       0.82      0.82      0.81     33779



  # n-gram = 2

## Application du bag of words en n-gram = 2

## Application d'un modèle de XGBOOOST sur les notes ( 0,1)

In [ ]:
# n-gram = 2
## Application du bag of words en n-gram = 2
## Application d'un modèle de XGBOOOST sur les notes ( 0,1)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(2,2) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=10000,learning_rate=0.01,max_depth=3,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en Bi-gram est est : ', round((t1-t0)/60, 2), 'minutes')
# Afficher l'accuracy sur le jeu de données de test.
print(' le score de xgboost classifier est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix


Vocabulary size:  4000
{'chaussures très': 583, 'très mauvaises': 3863, 'mauvaises qualité': 2101, 'livraison temp': 1919, 'temp emballage': 3581, 'emballage nickel': 1422, 'qualité produits': 2842, 'jaime bien': 1655, 'bien site': 385, 'rapide simple': 2933, 'tout bien': 3668, 'bien passé': 362, 'satisfaite produit': 3265, 'point relais': 2560, 'relais non': 3006, 'non disponible': 2232, 'cela fait': 523, 'deux fois': 1205, 'faire livrer': 1504, 'livrer colis': 1934}

le temps de calcul du Xgboost en Bi-gram est est :  6.44 minutes
 le score de xgboost classifier est : 0.78

              precision    recall  f1-score   support

         0.0       0.86      0.55      0.67     13516
         1.0       0.76      0.94      0.84     20263

    accuracy                           0.78     33779
   macro avg       0.81      0.75      0.76     33779
weighted avg       0.80      0.78      0.77     33779



Classe prédite,0,1
Classe réelle,,
0.0,7436,6080
1.0,1186,19077


# n-gram = 3

## Application du bag of words en n-gram = 3

## Application d'un modèle de RandomForest classification sur les notes ( 1,2,3,4,5)

In [ ]:
# n-gram = 2
# Application du bag of words en n-gram = 2
# Application d'un modèle de RandomForest classification sur les notes ( 1,2,3,4,5)

X = df.Commentaire

y = df.star##

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(3,3))

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

# appliquer un randomforest classifier
t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()


print('le temps de calcul du randomforest classification en tri-gram est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score du randomforest est :', round(rf.score(X_test, y_test),2))
print()

# Affichage d'un rapport de classification
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))


Vocabulary size:  4000
{'livraison temp emballage': 1944, 'jaime bien site': 1669, 'tout bien passé': 3582, 'bien passé merci': 303, 'problème point relais': 2615, 'point relais non': 2519, 'commandés bonne qualité': 961, 'bonne qualité délais': 446, 'très contente achat': 3815, 'showroom privé jamais': 3345, 'satisfaite commande reçu': 3217, 'commande reçu colis': 842, 'délais trop long': 1428, 'trop long frais': 3664, 'long frais port': 2032, 'frais port cher': 1596, 'rien bien méchant': 3092, 'déposé point relais': 1433, 'plus dun mois': 2419, 'colis point relais': 644}

le temps de calcul du randomforest classification en tri-gram est :  5.74 minutes

le score du randomforest est : 0.42

              precision    recall  f1-score   support

         1.0       0.50      0.30      0.37      4363
         2.0       0.23      0.05      0.09      3114
         3.0       0.38      0.19      0.26      6039
         4.0       0.46      0.19      0.27      8697
         5.0       0.42     

# n-gram = 3
# Application du bag of words en n-gram = 3
# Application d'un modèle de XGBOOOST sur les notes ( 0,1)

In [ ]:
# n-gram = 2
## Application du bag of words en n-gram = 2
## Application d'un modèle de XGBOOOST sur les notes ( 0,1)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(3,3) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=10000,learning_rate=0.01,max_depth=3,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en Tri-gram est est : ', round((t1-t0)/60, 2), 'minutes')
# Afficher l'accuracy sur le jeu de données de test.
print(' le score de xgboost classifier est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Vocabulary size:  4000
{'livraison temp emballage': 1944, 'jaime bien site': 1669, 'tout bien passé': 3582, 'bien passé merci': 303, 'problème point relais': 2615, 'point relais non': 2519, 'commandés bonne qualité': 961, 'bonne qualité délais': 446, 'très contente achat': 3815, 'showroom privé jamais': 3345, 'satisfaite commande reçu': 3217, 'commande reçu colis': 842, 'délais trop long': 1428, 'trop long frais': 3664, 'long frais port': 2032, 'frais port cher': 1596, 'rien bien méchant': 3092, 'déposé point relais': 1433, 'plus dun mois': 2419, 'colis point relais': 644}

le temps de calcul du Xgboost en Tri-gram est est :  3.71 minutes
 le score de xgboost classifier est : 0.68

              precision    recall  f1-score   support

         0.0       0.86      0.23      0.36     13516
         1.0       0.65      0.97      0.78     20263

    accuracy                           0.68     33779
   macro avg       0.75      0.60      0.57     33779
weighted avg       0.73      0.68    

Classe prédite,0,1
Classe réelle,,
0.0,3060,10456
1.0,517,19746


## n-gram = 1 à 2
## Application du bag of words en n-gram = 1 à 2 cumulés
## Application du modèle RandomForest sur les notes (0, 1)
## pour signifier respectivement, négatif (1* ,2* ,3* ) et positif(4* ,5* )


In [ ]:
# Application du modèle RandomForest sur les notes (0, 1)
# pour signifier respectivement, négatif (1*,2*,3*) et positif(4*,5*)

X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(1,2) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()

print('le temps de calcul du randomforest classification en Uni et Bi-gram est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score en randomforest est :', round(rf.score(X_test, y_test),2))
print()

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Vocabulary size:  4000
{'chaussures': 527, 'très': 3774, 'mauvaises': 2140, 'qualité': 2886, 'elles': 1310, 'trop': 3742, 'rapidement': 2966, 'chaussures très': 528, 'livraison': 1905, 'temp': 3607, 'emballage': 1312, 'nickel': 2292, 'marchandises': 2115, 'conformes': 830, 'parfait': 2408, 'livraison temp': 1974, 'recommande': 3004, 'achats': 22, 'retours': 3150, 'faciles': 1425}

le temps de calcul du randomforest classification en Uni et Bi-gram est :  6.27 minutes

le score en randomforest est : 0.87

              precision    recall  f1-score   support

         0.0       0.83      0.83      0.83     13516
         1.0       0.89      0.89      0.89     20263

    accuracy                           0.87     33779
   macro avg       0.86      0.86      0.86     33779
weighted avg       0.87      0.87      0.87     33779




Classe prédite,0.0,1.0
Classe réelle,,
0.0,11271,2245
1.0,2284,17979


# n-gram = 1 à 2
# Application du bag of words en n-gram = 1 à 2 cumulés
# Application d'un modèle de XGBOOOST sur les notes ( 0,1)

In [ ]:
# n-gram = 1 à 2
## Application du bag of words en n-gram = 1 à 2
## Application d'un modèle de XGBOOOST sur les notes ( 0,1)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(1,2) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))
print()

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=10000,learning_rate=0.01,max_depth=3,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en Uni-Bi gram cumulés  est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print(' le score de xgboost classifier est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Vocabulary size:  4000

{'chaussures': 527, 'très': 3774, 'mauvaises': 2140, 'qualité': 2886, 'elles': 1310, 'trop': 3742, 'rapidement': 2966, 'chaussures très': 528, 'livraison': 1905, 'temp': 3607, 'emballage': 1312, 'nickel': 2292, 'marchandises': 2115, 'conformes': 830, 'parfait': 2408, 'livraison temp': 1974, 'recommande': 3004, 'achats': 22, 'retours': 3150, 'faciles': 1425}

le temps de calcul du Xgboost en Uni-Bi gram cumulés  est :  17.34 minutes

 le score de xgboost classifier est : 0.86

              precision    recall  f1-score   support

         0.0       0.87      0.78      0.82     13516
         1.0       0.86      0.92      0.89     20263

    accuracy                           0.86     33779
   macro avg       0.86      0.85      0.86     33779
weighted avg       0.86      0.86      0.86     33779



Classe prédite,0,1
Classe réelle,,
0.0,10504,3012
1.0,1592,18671


# n-gram = 2-4
# Application du bag of words en n-gram = 2-4 cumulés
# Application d'un modèle de XGBOOOST sur les notes ( 0,1)

In [ ]:
# n-gram = 1 à 2
## Application du bag of words en n-gram = 1 à 2
## Application d'un modèle de XGBOOOST sur les notes ( 0,1)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(2,4) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))
print()

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=10000,learning_rate=0.01,max_depth=3,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en 2 et 4 gram cumulés  est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print(' le score de xgboost classifier est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Vocabulary size:  4000

{'chaussures très': 549, 'livraison temp': 1912, 'temp emballage': 3521, 'emballage nickel': 1406, 'qualité produits': 2835, 'jaime bien': 1634, 'bien site': 355, 'rapide simple': 2914, 'tout bien': 3616, 'bien passé': 327, 'tout bien passé': 3619, 'satisfaite produit': 3231, 'point relais': 2546, 'non disponible': 2240, 'cela fait': 492, 'deux fois': 1155, 'faire livrer': 1486, 'livrer colis': 1943, 'colis nest': 647, 'nest plus': 2215}

le temps de calcul du Xgboost en 2 et 4 gram cumulés  est :  6.52 minutes

 le score de xgboost classifier est : 0.78

              precision    recall  f1-score   support

         0.0       0.86      0.55      0.67     13516
         1.0       0.76      0.94      0.84     20263

    accuracy                           0.78     33779
   macro avg       0.81      0.75      0.76     33779
weighted avg       0.80      0.78      0.77     33779



Classe prédite,0,1
Classe réelle,,
0.0,7436,6080
1.0,1194,19069


# GridSearch en Xboost avec ngram=1 à 2 à 10000 features
# n-gram = 1 à 2
# Application du bag of words en n-gram = 1 à 2 cumulés
# Application d'un modèle de XGBOOOST sur les notes ( 0,1)

# n-gram = 1 à 2
## Application du bag of words en n-gram = 1 à 2
## Application d'un modèle de XGBOOOST sur les notes ( 0,1)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=2000 , stop_words = stopwords.words('french'), ngram_range=(1,2) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))
print()

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=1000,learning_rate=0.01,max_depth=6,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en Uni-Bi gram cumulés  est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print(' le score de xgboost classifier est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

## n-gram = 1 à 3
## Application du bag of words en n-gram = 1 à 3 cumulés
## Application du modèle RandomForest sur les notes (0, 1)
## pour signifier respectivement, négatif (1* ,2* ,3* ) et positif(4* ,5* )

In [ ]:
# Application du modèle RandomForest sur les notes (0, 1)
# pour signifier respectivement, négatif (1*,2*,3*) et positif(4*,5*)

X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(1,3) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()

print('le temps de calcul du randomforest classification en Bi-gram est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score en randomforest est :', round(rf.score(X_test, y_test),2))
print()

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Vocabulary size:  4000
{'chaussures': 527, 'très': 3754, 'mauvaises': 2153, 'qualité': 2890, 'elles': 1333, 'trop': 3721, 'rapidement': 2968, 'livraison': 1900, 'temp': 3577, 'emballage': 1335, 'nickel': 2306, 'marchandises': 2129, 'conformes': 835, 'parfait': 2414, 'livraison temp': 1983, 'recommande': 3006, 'achats': 21, 'retours': 3142, 'faciles': 1447, 'produits': 2783}

le temps de calcul du randomforest classification en Bi-gram est :  6.3 minutes

le score en randomforest est : 0.87

              precision    recall  f1-score   support

         0.0       0.83      0.83      0.83     13516
         1.0       0.89      0.89      0.89     20263

    accuracy                           0.87     33779
   macro avg       0.86      0.86      0.86     33779
weighted avg       0.87      0.87      0.87     33779




Classe prédite,0.0,1.0
Classe réelle,,
0.0,11263,2253
1.0,2258,18005


# n-gram = 1 à 3
# Application du bag of words en n-gram = 1 à 3
# Application d'un modèle de XGBOOOST sur les notes ( 0,1)

In [ ]:
# n-gram = 1 à 3
## Application du bag of words en n-gram = 1 à 3
## Application d'un modèle de XGBOOOST sur les notes ( 0,1)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(1,3) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))
print()

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=10000,learning_rate=0.01,max_depth=3,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en Uni-Bi-Tri gram cumulés  est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print(' le score de xgboost classifier est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Vocabulary size:  4000

{'chaussures': 527, 'très': 3754, 'mauvaises': 2153, 'qualité': 2890, 'elles': 1333, 'trop': 3721, 'rapidement': 2968, 'livraison': 1900, 'temp': 3577, 'emballage': 1335, 'nickel': 2306, 'marchandises': 2129, 'conformes': 835, 'parfait': 2414, 'livraison temp': 1983, 'recommande': 3006, 'achats': 21, 'retours': 3142, 'faciles': 1447, 'produits': 2783}

le temps de calcul du Xgboost en Uni-Bi-Tri gram cumulés  est :  17.49 minutes

 le score de xgboost classifier est : 0.86

              precision    recall  f1-score   support

         0.0       0.87      0.78      0.82     13516
         1.0       0.86      0.92      0.89     20263

    accuracy                           0.86     33779
   macro avg       0.86      0.85      0.86     33779
weighted avg       0.86      0.86      0.86     33779



Classe prédite,0,1
Classe réelle,,
0.0,10508,3008
1.0,1592,18671


## n-gram = 1 à 4
## Application du bag of words en n-gram = 1 à 4 cumulés
## Application du modèle RandomForest sur les notes (0, 1)
## pour signifier respectivement, négatif (1* ,2* ,3* ) et positif(4* ,5* )

In [ ]:
# Application du modèle RandomForest sur les notes (0, 1)
# pour signifier respectivement, négatif (1*,2*,3*) et positif(4*,5*)

X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(1,4) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

# Entrainer un modèle de Random Forest
rf = RandomForestClassifier(n_jobs=-1)
# Entraînement du modèle.
rf.fit(X_train, y_train)
t1 = time.time()

print('le temps de calcul du randomforest classification en Bi-gram est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print('le score en randomforest est :', round(rf.score(X_test, y_test),2))
print()

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print()
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Vocabulary size:  4000
{'chaussures': 528, 'très': 3752, 'mauvaises': 2162, 'qualité': 2894, 'elles': 1341, 'trop': 3719, 'rapidement': 2970, 'livraison': 1906, 'temp': 3577, 'emballage': 1343, 'nickel': 2313, 'marchandises': 2138, 'conformes': 834, 'parfait': 2419, 'livraison temp': 1991, 'recommande': 3008, 'achats': 21, 'retours': 3144, 'faciles': 1455, 'produits': 2786}

le temps de calcul du randomforest classification en Bi-gram est :  6.48 minutes

le score en randomforest est : 0.87

              precision    recall  f1-score   support

         0.0       0.83      0.83      0.83     13516
         1.0       0.89      0.89      0.89     20263

    accuracy                           0.87     33779
   macro avg       0.86      0.86      0.86     33779
weighted avg       0.87      0.87      0.87     33779




Classe prédite,0.0,1.0
Classe réelle,,
0.0,11281,2235
1.0,2264,17999


# n-gram = 1 à 4
# Application du bag of words en n-gram = 1 à 4
# Application d'un modèle de XGBOOOST sur les notes ( 0,1)

In [ ]:
# n-gram = 1 à 4
## Application du bag of words en n-gram = 1 à 4
## Application d'un modèle de XGBOOOST sur les notes ( 0,1)


X = df.Commentaire

y = df['star'].replace([1,2,3,4,5], [0,0,0,1,1])

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

vectorizer = CountVectorizer(max_features=4000 , stop_words = stopwords.words('french'), ngram_range=(1,4) )

X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# taille du vocabulaire
voc = vectorizer.vocabulary_
print('Vocabulary size: ', len(voc))
print()

# Affichage d'un échantilllon du vocabulaire
print({k: voc[k] for k in list(voc)[:20]})
print()

t0 = time.time()

xgb_clf = xgb.XGBClassifier(n_estimators=10000,learning_rate=0.01,max_depth=3,n_jobs=-1)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

t1 = time.time()


print('le temps de calcul du Xgboost en Uni-Bi-Tri gram cumulés  est : ', round((t1-t0)/60, 2), 'minutes')
print()
# Afficher l'accuracy sur le jeu de données de test.
print(' le score de xgboost classifier est :', round(xgb_clf.score(X_test, y_test),2))
print()

print( classification_report(y_test, y_pred) )

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

In [ ]:
end_programme = time.time()
print("le temps total d'exécution du programme est  ", round((end_programme - start_programme)/60, 2), 'minutes')